<a href="https://colab.research.google.com/github/lucevito/image/blob/main/random_forest_augmented.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive'
/content/drive/MyDrive


In [4]:
import os
import uuid
import glob
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, jaccard_score, confusion_matrix, f1_score
import tensorflow as tf
import os
import uuid
import glob
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras import *
import matplotlib.pyplot as plt
from keras.utils import plot_model
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support


encoder_filters = [256,512,1024]
decoder_filters = encoder_filters[::-1]
kernel = (2,2)
num_classes = 1
pool_size = (2, 2)

size = (32, 32)
canali_selezionati = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
input_shape = (size[0], size[1], len(canali_selezionati))
modello = 'modello.h5'


In [5]:
train_images_path = 'Immagini_satellitari/Train/images'
train_masks_path = 'Immagini_satellitari/Train/masks'
train_images_files = glob.glob(train_images_path + '/*.npy')
train_masks_files = glob.glob(train_masks_path + '/*.npy')
train_images_batch = np.array([np.load(file) for file in train_images_files])
train_masks_batch = np.array([np.load(file) for file in train_masks_files])
train_images_batch = train_images_batch[:, :, :, canali_selezionati]
train_masks_batch = tf.expand_dims(train_masks_batch, axis=-1)

test_images_path = 'Immagini_satellitari/Test/images'
test_masks_path = 'Immagini_satellitari/Test/masks'
test_images_files = glob.glob(test_images_path + '/*.npy')
test_masks_files = glob.glob(test_masks_path + '/*.npy')
test_images_batch = np.array([np.load(file) for file in test_images_files])
test_masks_batch = np.array([np.load(file) for file in test_masks_files])
test_images_batch = test_images_batch[:, :, :, canali_selezionati]
test_masks_batch = tf.expand_dims(test_masks_batch, axis=-1)

images_to_keep = []
masks_to_keep = []

for image, mask in zip(train_images_batch, train_masks_batch):
    if not np.all(mask == 0):
        images_to_keep.append(image)
        masks_to_keep.append(mask)

train_images_batch_filtered = np.array(images_to_keep)
train_masks_batch_filtered = np.array(masks_to_keep)
print(train_images_batch_filtered.shape)


(311, 32, 32, 10)


In [6]:
def flip_left_right(input_image, input_mask):
  channels = tf.split(input_image, num_or_size_splits=10, axis=-1)
  flipped_channels = [tf.image.flip_left_right(channel) for channel in channels]
  input_image = tf.concat(flipped_channels, axis=-1)
  input_mask = tf.image.flip_left_right(input_mask)
  return input_image, input_mask

# Rotazione
def rotate_image(input_image, input_mask, angle=45):
    channels = tf.split(input_image, num_or_size_splits=10, axis=-1)
    rotate_channels = [tf.image.rot90(channel, k=angle // 90) for channel in channels]

    input_image = tf.concat(rotate_channels, axis=-1)
    input_mask = tf.image.rot90(input_mask, k=angle // 90)
    return input_image, input_mask

# Zoom
def zoom_image(input_image, input_mask, zoom_factor=0.8):
    channels = tf.split(input_image, num_or_size_splits=10, axis=-1)
    zoom_channels = [tf.image.central_crop(channel, zoom_factor) for channel in channels]

    input_image = tf.concat(zoom_channels, axis=-1)
    input_mask = tf.image.central_crop(input_mask, zoom_factor)
    return input_image, input_mask

# Traslazione
def translate_image(input_image, input_mask, translation=(10, -5)):
    channels = tf.split(input_image, num_or_size_splits=10, axis=-1)
    translate_channels = [tf.roll(channel, shift=translation, axis=[0, 1]) for channel in channels]
    input_image = tf.concat(translate_channels, axis=-1)
    input_mask = tf.roll(mask, shift=translation, axis=[0, 1])
    return input_image, input_mask

# Cambiamento delle dimensioni
def resize_image(input_image, input_mask, new_size = (128,128)):
    channels = tf.split(input_image, num_or_size_splits=10, axis=-1)
    resize_channels = [tf.image.resize(channel,new_size) for channel in channels]
    input_image = tf.concat(resize_channels, axis=-1)
    input_mask = tf.image.resize(input_mask, new_size)
    return input_image, input_mask


def normalize(input_image, input_mask):
   input_image = tf.cast(input_image, tf.float32) / 255.0
#   input_mask -= 1
   return input_image, input_mask

In [7]:
def transform_image_train(input_image,input_mask):
  input_image, input_mask = normalize(input_image, input_mask)
  return input_image, input_mask

def transform_image_test(input_image,input_mask):
  input_image, input_mask = normalize(input_image, input_mask)
  return input_image, input_mask

In [8]:

augmented_images = []
augmented_masks = []

for image, mask in zip(train_images_batch, train_masks_batch):
  augmented_images.append(image)
  augmented_masks.append(mask)
  if not np.all(mask == 0):
    augmented_image,augmented_mask = flip_left_right(image,mask)
    augmented_images.append(augmented_image)
    augmented_masks.append(augmented_mask)

    augmented_image,augmented_mask = rotate_image(image,mask,15)
    augmented_images.append(augmented_image)
    augmented_masks.append(augmented_mask)

    augmented_image,augmented_mask = rotate_image(image,mask,30)
    augmented_images.append(augmented_image)
    augmented_masks.append(augmented_mask)
    augmented_image,augmented_mask = rotate_image(image,mask,45)
    augmented_images.append(augmented_image)
    augmented_masks.append(augmented_mask)

    augmented_image,augmented_mask = rotate_image(image,mask,60)
    augmented_images.append(augmented_image)
    augmented_masks.append(augmented_mask)
    augmented_image,augmented_mask = rotate_image(image,mask,75)
    augmented_images.append(augmented_image)
    augmented_masks.append(augmented_mask)

    augmented_image,augmented_mask = rotate_image(image,mask,90)
    augmented_images.append(augmented_image)
    augmented_masks.append(augmented_mask)

    augmented_image,augmented_mask = rotate_image(image,mask,120)
    augmented_images.append(augmented_image)
    augmented_masks.append(augmented_mask)

    augmented_image,augmented_mask = rotate_image(image,mask,180)
    augmented_images.append(augmented_image)
    augmented_masks.append(augmented_mask)

    augmented_image,augmented_mask = rotate_image(image,mask,270)
    augmented_images.append(augmented_image)
    augmented_masks.append(augmented_mask)


train_images_batch_augmented = augmented_images
train_masks_batch_augmented = augmented_masks

transformed_images_masks_train = [transform_image_test(image, mask) for image, mask in zip(train_images_batch_augmented, train_masks_batch_augmented)]
train_images = np.array([item[0] for item in transformed_images_masks_train])
train_masks = np.array([item[1] for item in transformed_images_masks_train])

transformed_images_masks_test = [transform_image_test(image, mask) for image, mask in zip(test_images_batch, test_masks_batch)]
test_images = np.array([item[0] for item in transformed_images_masks_test])
test_masks = np.array([item[1] for item in transformed_images_masks_test])

In [9]:
train_images, val_images, train_masks, val_masks = train_test_split(train_images,
                                                                    train_masks,
                                                                    test_size=0.2, random_state=42)

print('train_images :')
num_pixel_per_image = 32 * 32 * 10
train_images = train_images.reshape(-1, num_pixel_per_image)
print(train_images.shape)

print('train_masks :')
num_pixel_per_mask = 32 * 32 * 1
train_masks = train_masks[:, :32, :32, np.newaxis]
train_masks = train_masks.reshape(-1, num_pixel_per_mask)
print(train_masks.shape)

print('val_images :')
num_pixel_per_image = 32 * 32 * 10
val_images = val_images.reshape(-1, num_pixel_per_image)
print(val_images.shape)

print('val_masks :')
num_pixel_per_mask = 32 * 32 * 1
val_masks = val_masks[:, :32, :32, np.newaxis]
val_masks = val_masks.reshape(-1, num_pixel_per_mask)
print(val_masks.shape)


print('test_images :')
num_pixel_per_image = 32 * 32 * 10
test_images = test_images.reshape(-1, num_pixel_per_image)
print(test_images.shape)

print('test_masks :')
num_pixel_per_mask = 32 * 32 * 1
test_masks = test_masks[:, :32, :32, np.newaxis]
test_masks = test_masks.reshape(-1, num_pixel_per_mask)
print(test_masks.shape)

train_images :
(3011, 10240)
train_masks :
(3011, 1024)
val_images :
(753, 10240)
val_masks :
(753, 1024)
test_images :
(161, 10240)
test_masks :
(161, 1024)


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

rf_model = RandomForestClassifier(n_estimators=100,random_state=42)
rf_model.fit(train_images, train_masks)

val_predictions = rf_model.predict(val_images)
val_predictions = (val_predictions > 0.5).astype(np.uint8)

val_accuracy = accuracy_score(val_masks, val_predictions)
print("Validation Accuracy:", val_accuracy)

test_predictions = rf_model.predict(test_images)
test_predictions = (test_predictions > 0.5).astype(np.uint8)

test_accuracy = accuracy_score(test_masks, test_predictions)
print("Test Accuracy:", test_accuracy)

Validation Accuracy: 0.7051792828685259
Test Accuracy: 0.5652173913043478


In [11]:
print(test_masks.shape)
print(test_predictions.shape)

(161, 1024)
(161, 1024)


In [12]:
print(test_masks)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [13]:
print(test_predictions)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [14]:
test_masks = test_masks.flatten()
test_predictions = test_predictions.flatten()
print(test_masks.shape)
print(test_predictions.shape)

(164864,)
(164864,)


In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Calcola l'accuracy
accuracy = accuracy_score(test_masks, test_predictions)
print("Accuracy:", accuracy)

# Calcola la precision
precision = precision_score(test_masks, test_predictions)
print("Precision:", precision)

# Calcola il recall
recall = recall_score(test_masks, test_predictions)
print("Recall:", recall)

# Calcola l'F1-score
f1 = f1_score(test_masks, test_predictions)
print("F1-score:", f1)

# Calcola la confusion matrix
conf_matrix = confusion_matrix(test_masks, test_predictions)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.9832286005434783
Precision: 0.0
Recall: 0.0
F1-score: 0.0
Confusion Matrix:
[[162099      1]
 [  2764      0]]
